# Exemplo Detalhado de Uso - Cluster Fácil

Este notebook demonstra os passos essenciais e funcionalidades adicionais da biblioteca `cluster-facil`.

## 1. Instalação

Se você ainda não instalou a biblioteca, descomente e execute a célula abaixo. Ela instalará a versão mais recente diretamente do repositório GitHub.

In [ ]:
# !pip install git+https://github.com/bdcdo/cluster-facil.git
# 
# # Instale também dependências opcionais se for usar:
# !pip install matplotlib openpyxl pyarrow

## 2. Importações e Dados de Exemplo

Vamos importar o `pandas` para criar nosso DataFrame de exemplo e a classe `ClusterFacil` da nossa biblioteca.

In [ ]:
import pandas as pd
from cluster_facil import ClusterFacil

# Criando um DataFrame de exemplo simples com textos
data = {'id': [1, 2, 3, 4, 5, 6, 7, 8],
        'texto_documento': [
            "O juiz determinou o pagamento da indenização.",
            "Recurso de apelação foi interposto pela defesa.",
            "A sentença julgou procedente o pedido do autor.",
            "A corte negou provimento ao recurso especial.",
            "Processo arquivado por falta de movimentação.",
            "Acordo homologado entre as partes.",
            "Indenização por danos morais foi concedida.",
            "Extinção do processo sem resolução de mérito."
        ]}

In [ ]:
df_exemplo = pd.DataFrame(data)

print("DataFrame de Exemplo:")
display(df_exemplo)

## 3. Inicializando o ClusterFacil

Criamos uma instância da classe, passando nosso DataFrame.
Usamos `random_state=42` para garantir que os resultados da clusterização sejam reprodutíveis.

In [ ]:
cf = ClusterFacil(df_exemplo, random_state=42)

## 4. Preparando os Dados e Analisando o Cotovelo

Chamamos o método `preparar`. Ele fará o pré-processamento (TF-IDF) e **exibirá o gráfico do método do cotovelo** (se `matplotlib` estiver instalado e `plotar_cotovelo=True`, que é o padrão).

Precisamos informar:
*   `coluna_textos`: O nome da coluna que contém os textos.
*   `limite_k` (opcional): O número máximo de clusters (K) a testar no método do cotovelo (padrão é 10).

In [ ]:
print("Iniciando a preparação e geração do gráfico do cotovelo...")
cf.preparar(coluna_textos='texto_documento', limite_k=5)
print("Preparação concluída.")

### Analisando o Gráfico do Cotovelo

Olhe o gráfico gerado acima. O eixo Y mostra a inércia (WCSS - Within-Cluster Sum of Squares) e o eixo X mostra o número de clusters (K).

Procure por um "cotovelo" (elbow) na curva: um ponto onde a taxa de diminuição da inércia desacelera significativamente. Adicionar mais clusters depois desse ponto geralmente não traz um ganho tão grande na separação dos grupos.

**Neste exemplo, o cotovelo parece estar em K=3.**

## 5. Clusterizando

Agora que escolhemos nosso K (K=3), vamos executar a clusterização com o método `clusterizar`.

In [ ]:
k_escolhido = 3
print(f"Executando a clusterização com K={k_escolhido}...")
nome_coluna_cluster = cf.clusterizar(num_clusters=k_escolhido)
print(f"Clusterização concluída. Coluna '{nome_coluna_cluster}' adicionada ao DataFrame.")

## 6. Verificando o Resultado da Clusterização

O DataFrame original dentro do objeto `cf` agora possui a nova coluna (ex: `cluster_1`) com o ID do cluster atribuído a cada documento.

In [ ]:
print("DataFrame com a coluna de cluster:")
display(cf.df[['id', nome_coluna_cluster, 'texto_documento']])

## 7. Salvando os Resultados

Podemos salvar o DataFrame completo e/ou amostras de cada cluster usando o método `salvar`.

In [ ]:
print("Salvando os resultados...")
status_salvar = cf.salvar(
    o_que_salvar='ambos',      # Salva o DF completo e as amostras
    formato_tudo='csv',        # Salva o DF completo como CSV
    formato_amostras='xlsx',   # Salva as amostras como Excel (requer openpyxl)
    # caminho_tudo='output/meu_df_completo.csv', # Opcional: caminho específico
    # caminho_amostras='output/minhas_amostras.xlsx', # Opcional: caminho específico
    # diretorio_saida='output/' # Opcional: diretório para nomes padrão
)

print("\nStatus do salvamento:")
print(status_salvar)

if status_salvar.get('tudo_salvo'):
    print(f"- DataFrame completo salvo em: {status_salvar.get('caminho_tudo')}")
if status_salvar.get('amostras_salvas'):
    print(f"- Amostras salvas em: {status_salvar.get('caminho_amostras')}")

## 8. Classificando Clusters

Após analisar as amostras ou o DataFrame completo, podemos atribuir rótulos (classificações) aos clusters usando o método `classificar`.

Isso cria/atualiza a coluna definida em `nome_coluna_classificacao` (padrão: 'classificacao').

In [ ]:
# Supondo que, após análise, o cluster 0 e 2 são 'Procedente/Indenização'
# e o cluster 1 é 'Recurso/Extinção'
cf.classificar(cluster_ids=[0, 2], classificacao='Procedente/Indenização')
cf.classificar(cluster_ids=1, classificacao='Recurso/Extinção')

print("DataFrame com a coluna de classificação:")
# Exibindo colunas relevantes
display(cf.df[['id', nome_coluna_cluster, cf.nome_coluna_classificacao, 'texto_documento']])

## 9. Listando e Contando Classificações

Podemos ver as classificações únicas e suas contagens.

In [ ]:
print("Classificações únicas:", cf.listar_classificacoes())
print("\nContagem por classificação:")
print(cf.contar_classificacoes())

## 10. Criando um Subcluster

Se quisermos analisar mais a fundo um grupo específico (uma classificação), podemos criar um subcluster.
O método `subcluster` retorna uma **nova instância** de `ClusterFacil`, contendo apenas os dados da classificação desejada e pronta para um novo ciclo de `preparar` -> `clusterizar`.

In [ ]:
classificacao_alvo = 'Procedente/Indenização'
print(f"Criando subcluster para '{classificacao_alvo}'...")
sub_cf = cf.subcluster(classificacao_alvo)

print(f"Nova instância 'sub_cf' criada com {len(sub_cf.df)} linhas.")
print("DataFrame do subcluster (observe a coluna 'classificacao_origem'):")
display(sub_cf.df.head())

### Clusterizando o Subcluster

Agora podemos aplicar o processo novamente, mas apenas nos dados do subcluster.

In [ ]:
print("Preparando o subcluster...")
# Usamos a mesma coluna de texto original. limite_k pode ser menor.
# O gráfico do cotovelo será exibido para o subcluster.
sub_cf.preparar(coluna_textos='texto_documento', limite_k=3, plotar_cotovelo=True)

# Supondo que o cotovelo para o subcluster sugira K=2
k_subcluster = 2
print(f"\nClusterizando o subcluster com K={k_subcluster}...")
nome_coluna_subcluster = sub_cf.clusterizar(num_clusters=k_subcluster)

print(f"Subclusterização concluída. Coluna '{nome_coluna_subcluster}' adicionada ao DataFrame 'sub_cf.df'.")
display(sub_cf.df[['id', nome_coluna_subcluster, 'texto_documento']])

## 11. Resetando o Estado

Se quisermos recomeçar a análise no DataFrame original (por exemplo, tentar um K diferente na primeira rodada) sem perder os dados originais, podemos usar `resetar`.

Isso remove todas as colunas de cluster (`cluster_*`, `subcluster_*`) e a coluna de classificação (`classificacao`) da instância original `cf`.

In [ ]:
print("Estado do DataFrame 'cf' ANTES do reset:")
display(cf.df.head())

cf.resetar()

print("\nEstado do DataFrame 'cf' DEPOIS do reset (colunas de cluster/classificação removidas):")
display(cf.df.head())

--- 

Fim do exemplo detalhado!